# <b>Denison CS181/DA210 Final Project</b> 
## Deliverable #2: Data Acquisition
#### Cheryl Nguyen, Minh Le
#### Dr. Amert
#### April 5th, 2023

# <font color='pink'><b>Part 1: Introduction</b></font>

Whether there is a relationship between the Gross Domestic Product (GDP) per capita and the life expectancy of citizens has been the subject of debate for numerous people, including policymakers, researchers, and even philosophers, for years. Some people may concede that economic growth directly affects human well-being due to the healthcare services provided, but it is difficult to conclude whether a higher GDP per capita can lead to an overall increase in life expectancy. Recognizing the intriguing nature and practical application of this issue, we aim to settle it by conducting this research, as we will answer the following central question: 
##### <b>Does GDP per capita have a relationship with the life expectancy of countries in the world?</b>

To resolve this question, we make use of 2 datasets obtained from Wikipedia and The World Bank. The Life Expectancy dataset from Wikipedia provides data regarding the average life expectancy at birth of roughly 248 countries in the world in 2021. Meanwhile, the dataset from The World Bank shows the GDP per capita if countries in 2021. Our main agenda is to compare the difference between the group with high life expectancy and one with low life expectancy, thereby discovering whether there is any connection between GDP per capita and life expectancy, which is the answer to our central question.

The sources of the datasets will be given at the end of this notebook.

#### <b>Reasons for Eligibility</b>:

- The life expectancy dataset is taken from Wikipedia. According to the Copyrights section in Terms of Use (https://en.wikipedia.org/wiki/Wikipedia:Copyrights#Guidelines_for_images_and_other_media_files), texts on Wikipedia are co-licensed under the Creative Commons Attribution-ShareAlike 3.0 Unported License (CC BY-SA) and the GNU Free Documentation License (GFDL). These licenses allow users to have free access to information stored in Wikipedia, and in turn, they can copy, modify, and redistribute it.

- As regards the GDP per capita dataset, we have inspected the Data Access and Licensing page of The World Bank (https://datacatalog.worldbank.org/public-licenses#cc-by) and learned that the database provided on this website earns the Creative Commons Attribution 4.0 (CC-BY 4.0) International license. By integrating this license, the provider enables web users to “copy, modify and distribute data in any format for any purpose, including commercial use.” (The World Bank) To increase the certainty, we also referred to the overall Terms of Use (https://www.worldbank.org/en/about/legal/terms-of-use-for-datasets), which claims to allow users to “extract, download, and make copies of the data contained in the Datasets.” In other words, we can use the data in this course, which is for educational purposes. 

- The csv file named `countries.csv` is taken from the website of `denison.edu` (http://datasystems.denison.edu/data.html) under the `Tabular Data` section. This file is eligible to use because it originates from an education website owned by Denison University. Indeed, the location of the URL is `denison.edu`. As we are using this file for pure educational purposes in a project assigned by a Denison professor during school year, we have the ability to use it and modify it within the scope of this project only.


---
---

# <font color='orange'><b>Part 2: Data Acquisition</b></font>

In this phase of the project, we will use web scraping to extract one table of data before converting it to `pandas` DataFrame and acquire data from the remaining dataset and convert to a `pandas` DataFrame. We will briefly describe the steps throughout this notebook.

---

#### <b>Note</b>
The XML file downloaded from The World Bank is originally `API_NY.GDP.PCAP.CD_DS2_en_xml_v2_5353247.xml`. However, we find that this name is too lengthy, so we change it to `gdp.xml` for easy reference.

---

In [1]:
import os
import io
import sys
import importlib
import pandas as pd
from lxml import etree
import requests
from IPython.display import Image
import os.path

htmlparser =  etree.HTMLParser()

module_dir = os.path.join("..", "..", "modules")
module_path = os.path.abspath(module_dir)
if not module_path in sys.path:
    sys.path.append(module_path)

import util
importlib.reload(util)

import json
import sqlalchemy as sa

datadir = "data"

%load_ext sql

---
### <b>1. Web Scraping a Wikipedia Table and Converting to `pandas` DataFrame</b>

First of all, we will use web scraping to extract a `Wikipedia` table showing life expectancy at birth of countries in 2021, then convert it to a DataFrame using the `pandas` module.

The information about this dataset is as followed: 
- Description: The life expectancy of the world taken in 2021 from the United Nations
- Size: 248 rows and 5 columns. 
- Column names: <b>Countries and Region</b> (char), <b>All</b> (float), <b>M</b> (float), <b>F</b> (float), <b>Gender life gap</b> (float)
- Entity: Life expectancy
- Format: HTML file

First, we will use the `curl` command to download the webpage content to a local HTML file.  The following command will save the HTML source of [https://en.wikipedia.org/api/rest_v1/page/html/List_of_countries_by_life_expectancy#CIA_World_Factbook_(2014-2020)](https://en.wikipedia.org/api/rest_v1/page/html/List_of_countries_by_life_expectancy#CIA_World_Factbook_(2014-2020)) to our computer, in a file `life.html` in the same folder as this notebook.

In [2]:
!curl -s -o life.html https://en.wikipedia.org/api/rest_v1/page/html/List_of_countries_by_life_expectancy%23CIA_World_Factbook_(2014-2020)

zsh:1: unknown file attribute: 2


#### <b>Comment</b>: 
The `curl` demand cannot be executed successfully, so I resort to web scraping by web requests. I will use the Wikipedia API as the resource. The page [https://en.wikipedia.org/api/rest_v1/page/html/List_of_countries_by_life_expectancy#CIA_World_Factbook_(2014-2020)](https://en.wikipedia.org/api/rest_v1/page/html/List_of_countries_by_life_expectancy#CIA_World_Factbook_(2014-2020)) will provide access to a simpler version of the original Wikipedia page https://en.wikipedia.org/wiki/List_of_countries_by_life_expectancy#World_Health_Organization_(2019). I have also downloaded the HTML file of this source directly from the Internet and stored it in the `data` folder under the name `life.html`.

In [3]:
# Read HTML from the web
resource_path = "api/rest_v1/page/html/List_of_countries_by_life_expectancy#CIA_World_Factbook_(2014-2020)"

url = util.buildURL(resource_path, "en.wikipedia.org")
response = requests.get(url)
assert response.status_code == 200

# Use a custom HTML parser to parse the response content into an XML Element
life = etree.parse(io.BytesIO(response.content), htmlparser)
life_root = life.getroot()

There are multiple tables in this dataset, and we only pick 1 that represents data in 2021 to analyze.

In [4]:
# Find the needed table using a relevant attribute
table = life_root.xpath('//table/caption[text() = "Countries and regions by life expectancy at birth in 2021 (2022 report)"]/..')

# Get table root
tableroot = table[0]

# Display the first 20 lines of the tree
util.print_xml(tableroot, depth=16, nlines=20)

<table class='wikitable sortable static-row-numbers plainro
  <caption id='mwCgc'>Countries and regions by life expecta
    <sup ...>
      <a href='./List_of_countries_by_life_expectancy#cite_
        <span ...>[6]</span>
      </a>
    </sup>
  </caption>
  <tbody id='mwCgo'>
    <tr class='static-row-header' style='vertical-align:bot
      <th rowspan='2' id='mwCgw'>Countries and regions</th>
      <th colspan='4' id='mwCg0'>Life expectancy at birth</
    </tr>
    <tr class='static-row-header' style='vertical-align:bot
      <th id='mwCg8'>All</th>
      <th id='mwChA'>
        <abbr title='Male' about='#mwt348' typeof='mw:Trans
      </th>
      <th id='mwChI'>
        <abbr title='Female' about='#mwt351' typeof='mw:Tra


Subsequently, we will extract column names from the table and store them in the variable `col_names` by the use of xpath.

<b>Note</b>: As we use xpath, we see some inappropriate values. Hence, we will use list comprehension to solve this issue.

In [5]:
# Obtain the column names from the table
col_names = tableroot.xpath(".//th//text()")

# Use list comprehension to remove irrelevant values
col_names = [col_names[i] for i in range(len(col_names[:6])) if i != 1]

# Display column names
col_names

['Countries and regions', 'All', 'M', 'F', 'Gender life gap']

While collecting data using web requests, we find several unrelated values due to the intervention of some `th` tags in between `tr`, which makes both `th` and `td` children of some `tr`. These rows involve category names such as `Asia` or `High Income countries`, which are not country or region names. As a result, we will use procedural operations to add rows in succession into a LoL.

In [6]:
# Initialize an LoL
LoL = []

for child in tableroot.find("tbody").findall("tr"):
    if child.find("th") is not None: # Ignore any rows having both th and tr
        continue
    else:
        # Rows of country names have a child 'a' in td, while other rows do not 
        a = [val.text if val.find("a") is None else val.find("a").text for val in child.iter("td")]
    LoL.append(a)

Finally, we will build a `pandas` DataFrame from the LoL and column names.

In [7]:
life_df = pd.DataFrame(LoL, columns=col_names)
life_df.head(10)

,Countries and regions,All,M,F,Gender life gap
0,Afghanistan,62.0,58.9,65.3,6.4
1,Albania,76.5,74.1,79.2,5.1
2,Algeria,76.4,74.9,78.0,3.1
3,American Samoa,72.5,69.1,76.6,7.5
4,Andorra,80.4,77.2,84.3,7.1
5,Angola,61.6,59.0,64.3,5.3
6,Anguilla,76.6,73.2,80.5,7.3
7,Antigua and Barbuda,78.5,75.8,80.9,5.1
8,Argentina,75.4,72.2,78.6,6.4
9,Armenia,72.0,66.6,77.4,10.8


Above shows the first 10 rows of our DataFrame. In Deliverable #3, we plan to use the following columns: 
- `Countries and regions`: names of countries and regions (str)
- `F`: female's life expectancy at birth (float)
- `M`: male's life expectancy at birth (float)

---
### <b>2. Acquiring the GDP Per Capita Table and Converting to `pandas` DataFrame</b>

Secondly, we will acquire the data on GDP per capita in 2021 from `The World Bank` by utilizing the XML parser and procedural operations. Then, we we will convert it to a DataFrame by the `pandas` module.

The information about this dataset is as followed:
- Description: The GDP per capita of countries and territories in the world in 2021
- Size: 278 rows and 3 columns
- Column names: <b>Country</b> (char), <b>Most Recent Year</b> (int), <b>Most Recent Value</b> (float)
- Entity: GDP per capita
- Format: XML file 


In [8]:
# Use XML parser to parse data
path = os.path.join(datadir, 'gdp.xml')
myparser = etree.XMLParser(remove_blank_text=True)
tree = etree.parse(path, myparser)

# Obtain the root
gdp_root = tree.getroot()

##### <b>Note</b>:
As we look into `gdp.xml`, we observe that for each country or area, there are 4 fields corresponding to 4 attribute values. However, in the version displayed on the website, there are only 3 columns presented. The hidden column is represented by the attribute whose value is `Item` in the XML file. The text of this tag is `GDP per capita (current US$)`.

To include this column, we will create a DataFrame containing 4 columns: <b>Country or Area</b>, <b>Item</b>, <b>Year</b>, <b>Value</b>.

In [9]:
# Extract column names
col_names_gdp = gdp_root.xpath("//record[position()=1]/field/@name")

# Initialize DoL
DoL = {key: [] for key in col_names_gdp}

# Iterate through all the records in the XML
for record in gdp_root.findall(".//record"):
    # Add the texts of each field to corresponding lists
    country = record.find("./field[@name='Country or Area']").text
    item = record.find("./field[@name='Item']").text
    year = record.find("./field[@name='Year']").text
    value = record.find("./field[@name='Value']").text
    
    # Append the values to their respective lists
    DoL["Country or Area"].append(country)
    DoL["Item"].append(item)
    DoL["Year"].append(year)
    DoL["Value"].append(value)

# Create a pandas DataFrame from the lists
df = pd.DataFrame(DoL)

# Display the DataFrame
df.head(10)


,Country or Area,Item,Year,Value
0,Aruba,GDP per capita (current US$),1960,None
1,Aruba,GDP per capita (current US$),1961,None
2,Aruba,GDP per capita (current US$),1962,None
3,Aruba,GDP per capita (current US$),1963,None
4,Aruba,GDP per capita (current US$),1964,None
5,Aruba,GDP per capita (current US$),1965,None
6,Aruba,GDP per capita (current US$),1966,None
7,Aruba,GDP per capita (current US$),1967,None
8,Aruba,GDP per capita (current US$),1968,None
9,Aruba,GDP per capita (current US$),1969,None


In [10]:
# Filter DataFrame to acquire data in 2021
gdp_per_capita_df = df[df.Year=='2021'].copy()

gdp_per_capita_df.head(10)

,Country or Area,Item,Year,Value
61,Aruba,GDP per capita (current US$),2021,29342.1008575886
123,Africa Eastern and Southern,GDP per capita (current US$),2021,1549.77272965144
185,Afghanistan,GDP per capita (current US$),2021,368.754614175459
247,Africa Western and Central,GDP per capita (current US$),2021,1757.03062622138
309,Angola,GDP per capita (current US$),2021,1953.53375721508
371,Albania,GDP per capita (current US$),2021,6492.87201224634
433,Andorra,GDP per capita (current US$),2021,42137.3272710372
495,Arab World,GDP per capita (current US$),2021,6271.31900080308
557,United Arab Emirates,GDP per capita (current US$),2021,44315.5541834116
619,Argentina,GDP per capita (current US$),2021,10636.1201956183


Above shows the first 10 rows of our DataFrame. In Deliverable #3, we plan to use the following columns: 
- `Country or Area`: names of countries and areas (str)
- `Value`: GDP per capita in 2021 (float)

### <b>3. Countries Dataset</b>

We will acquire this dataset from `countries.csv` file that we have encountered (http://datasystems.denison.edu/data.html). This data will be read using `read_csv` and converted to a `pandas` DataFrame.

The information about this dataset is as followed:
- Description: The regions and income status of world countries.
- Size: 217 rows and 5 columns
- Column names: `code` (char), `name` (char), `currency` (char), `region` (char), `income` (char)
- Entity: countries
- Format: CSV file 

In [11]:
# Read in the file to convert to DataFrame

f = os.path.join(datadir, 'countries.csv')
countries_df = pd.read_csv(f)
countries_df.head(10)

,code,name,currency,region,income
0,ABW,Aruba,Aruban florin,Latin America & Caribbean,High income
1,AFG,Afghanistan,Afghan afghani,South Asia,Low income
2,AGO,Angola,Angolan kwanza,Sub-Saharan Africa,Lower middle income
3,ALB,Albania,Albanian lek,Europe & Central Asia,Upper middle income
4,AND,Andorra,Euro,Europe & Central Asia,High income
5,ARE,United Arab Emirates,U.A.E. dirham,Middle East & North Africa,High income
6,ARG,Argentina,Argentine peso,Latin America & Caribbean,Upper middle income
7,ARM,Armenia,Armenian dram,Europe & Central Asia,Upper middle income
8,ASM,American Samoa,U.S. dollar,East Asia & Pacific,Upper middle income
9,ATG,Antigua and Barbuda,East Caribbean dollar,Latin America & Caribbean,High income


# <font color='violet'><b>Part 3: Tidy Data and Database Design</b></font>

In this part, we will tidy data so that the data conforms to the database design principles.

## <b>1. Tidy Data</b>

As we work on tidying the datasets, we acknowledge several points that make them untidy and solutions to these issues: 
- Some countries are not assigned identical names in `gdp_xml`, `countries_csv`, and `life.html` (e.g: The Bahamas vs. Bahamas, The), so we have to manually find these inconsistencies and modify them to sync all tables.
- Both `Item` and `Value` columns in `gdp_per_capita_df` have generic column names: The only value in `Item` is `GDP per capita (current US$)` and the `Value` item should be renamed `GDP Per Capita` to precisely represent the meaning of values.
- We only use year 2021, so we will omit the `Year` column in `life_df`.
- As we observe `life_df`, we are aware that `Gender life gap` can be derived from `M` and `F` by taking the absolute value of the substraction and `All` can also be derived from `M` and `F`. Thus, we will only maintain the `All` column and change the name of this column.
- As regards `gdp_per_capita_df`, a number of rows in the `Country or Area` column do not represent country names (e.g: `World, Upper Middle Income`). Hence, we will use `right join` to join `gdp_per_capita` with `life_df` (only take the rows from `life_df` as all rows in this DataFrame signify country names).
- The 2 DataFrames `life_df` and `gdp_per_capita_df` have the same set of independent variable `Countries and Regions` and `Country or Area`. The wordings are different but the contents are the same, as they both represent names of countries and areas in the world. Hence, we will combine them into 1 table with the functional dependency: `Country and Area` -> `GDP Per Capita, All`. We will later rename the column names for consistency.
- We will drop columns `code`, `currency`, and `income` in `countries_df` because these are redundant information. We will also rename the column `name` into `country`.

In [12]:
# Drop columns
gdp_per_capita_df = gdp_per_capita_df.drop(['Item', 'Year'], axis=1)

# Rename columns
gdp_per_capita_df.rename(columns={'Country or Area': 'country_and_area', 'Value': 'gdp_per_capita'}, inplace=True)

# Set index 
gdp_per_capita_df.set_index('country_and_area', inplace=True)


In [13]:
# Drop columns
life_df = life_df.drop(['F', 'M', 'Gender life gap'], axis=1)

# Rename columns
life_df.rename(columns={'Countries and regions': 'country_and_area', 'All': 'life'}, inplace=True)

# Set index 
life_df.set_index('country_and_area', inplace=True)


In [14]:
# Drop columns
countries_df = countries_df.drop(['code', 'currency', 'income'], axis=1)

# Rename columns
countries_df.rename(columns={'name': 'country_and_area'}, inplace=True)

# Set index
countries_df.set_index('country_and_area', inplace=True)

In [15]:
# Join two tables
life_gdp = gdp_per_capita_df.join(life_df, how = 'right')

# Display dataset
life_gdp

,gdp_per_capita,life
country_and_area,,
Afghanistan,368.754614175459,62.0
Albania,6492.87201224634,76.5
Algeria,3690.62787797599,76.4
American Samoa,15743.3107582991,72.5
Andorra,42137.3272710372,80.4
...,...,...
Wallis and Futuna,NaN,78.4
Western Sahara,NaN,70.8
Yemen,NaN,63.8


#### <b>Note</b> 

We see that the `life_gdp` table and the `countries_df` table have the same independent variable (country and area). Thus, we will join two tables with a `left join` that only takes the rows from `countries_df`. Indeed, in the `life_gdp` table, several values in `country_and_area` column are island territories (e.g: `Angulia, or Mayotte`), which do not appear in `countries_df`. We only attain the rows from `countries_df` so as to remove some outliers for our analysis (island-based territories observe a low population, so it is bound to be difficult to establish a relationship between GDP per capita and life expectancy based on these data).

In [16]:
# Join two tables
indicators = countries_df.join(life_gdp, how='left')

# Reset index
indicators.reset_index(inplace=True)

# Display the dataset
indicators

,country_and_area,region,gdp_per_capita,life
0,Aruba,Latin America & Caribbean,29342.1008575886,74.6
1,Afghanistan,South Asia,368.754614175459,62.0
2,Angola,Sub-Saharan Africa,1953.53375721508,61.6
3,Albania,Europe & Central Asia,6492.87201224634,76.5
4,Andorra,Europe & Central Asia,42137.3272710372,80.4
...,...,...,...,...
212,Kosovo,Europe & Central Asia,5269.78390114389,76.8
213,Yemen,Middle East & North Africa,NaN,63.8
214,South Africa,Sub-Saharan Africa,7055.04477598783,62.3
215,Zambia,Sub-Saharan Africa,1137.34363269164,61.2


## <b>2. Database Design</b>

We will present information concerning our database design and tactics to tidy the data before using SQL queries to create tables.


### <b>a. Table Design</b>

The original datasets have now been combined to 1 table named `indicators`.

### <b>b. Table-Field Design</b>

4 fields: `country and area` (primary key), `region`, `gdp per capita`, `life`.

### <b>c. Sound Database Design</b>

1. Tables name: `indicators`: plural name. 
2. Primary key: `country_and_area` as a singleton key.
3. Functional dependency: `country_and_area` -> `region, gdp per capita, life`. 


#### Reasons for Adherence to Principles of Good Database Design:
- There are no values as fields.
- There are no mashups.
- Several fields from the original tables have been dropped and 2 tables have been combined into 1 -> no redundant information.
- There are no derived fields.

# <font color='turquoise'><b>Part 4: SQL Queries</b></font>

For his part, we will execute SQL queries to create tables in our database and populate them using the data parsed from our data sources.

## <b>1. Create Table</b>

As we have narrowed original tables down to only 1, we will make 1 set of SQL statements to create it.

First, we will use 2 functions to set credentials and build a connection string. This connection string, called `cstring`, rolls together the information regarding the scheme, the directory containing the database files, and the name of the database.  

In [17]:
def getsqlite_creds(dirname=".",filename="creds.json",source="sqlite"):
    """ Using directory and filename parameters, open a credentials file
        and obtain the two parts needed for a connection string to
        a local provider using the "sqlite" dictionary within
        an outer dictionary.  
        
        Return a scheme and a dbfile
    """
    assert os.path.isfile(os.path.join(dirname, filename))
    with open(os.path.join(dirname, filename)) as f:
        D = json.load(f)
    sqlite = D[source]
    return sqlite["scheme"], sqlite["dbdir"], sqlite["database"]


In [18]:
def buildConnectionString(source="sqlite_country"):
    scheme, dbdir, database = getsqlite_creds(source=source)
    template = '{}:///{}/{}.db'
    return template.format(scheme, dbdir, database)

Next, we will construct a connection string to connect to our `country` database.

In [19]:
# Build the conection string
cstring = buildConnectionString("sqlite_country")
print("Connection string:", cstring)

# Connect to the database
engine = sa.create_engine(cstring)
connection = engine.connect()

Connection string: sqlite:///./country.db


Next, we will create the table `indicators` on SQLiteStudio.

In [20]:
query = """
CREATE TABLE IF NOT EXISTS indicators (country_and_area VARCHAR(64) NOT NULL PRIMARY KEY, 
              region VARCHAR(64) NULL,
              gdp_per_capita FLOAT NULL, 
              life FLOAT NULL)
              
"""
# Execute table creation
try:
    connection.execute(query)
except sa.exc.SQLAlchemyError as err:
    print("CREATE of indicators failed:", str(err))

## <b>2. Populate Table</b>

Then, we will populate the table by inserting data from `indicators`.

In [21]:
indicators.to_sql("indicators", con=connection, if_exists="append", index=False)

217

Now, we will create a DataFrame called `indicators_df` from the table to check if data have been populated.

In [22]:
indicators_df = pd.read_sql_table("indicators", connection)
indicators_df.head()

,country_and_area,region,gdp_per_capita,life
0,Aruba,Latin America & Caribbean,29342.100858,74.6
1,Afghanistan,South Asia,368.754614,62.0
2,Angola,Sub-Saharan Africa,1953.533757,61.6
3,Albania,Europe & Central Asia,6492.872012,76.5
4,Andorra,Europe & Central Asia,42137.327271,80.4


In [23]:
# Close the connection
try:
    connection.close()
except:
    pass
del engine

---
---
### <b>References</b>
1. gdp.xml: https://data.worldbank.org/indicator/NY.GDP.PCAP.CD?end=2019&start=1960

2. life.html: https://en.wikipedia.org/wiki/List_of_countries_by_life_expectancy#World_Health_Organization_(2019)

3. countries.csv: http://datasystems.denison.edu/data.html